In [4]:
import os

# 读取所有环境变量
env_vars = os.environ

# 打印每个环境变量的名称和值
for var, value in env_vars.items():
    print(f'{var}: {value}')

ACSETUPSVCPORT: 23210
ACSVCPORT: 17532
ALLUSERSPROFILE: C:\ProgramData
ANACONDA_HOME: C:\ProgramData\anaconda3
APPCODE_VM_OPTIONS: C:\Program Files\JetBrains\jetbra\vmoptions\appcode.vmoptions
APPDATA: C:\Users\Administrator\AppData\Roaming
APPLICATIONINSIGHTS_CONFIGURATION_CONTENT: {}
APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL: true
BETTERNCM_PROFILE: C:\Users\Administrator\BetterNCM
CHOCOLATEYINSTALL: C:\ProgramData\chocolatey
CHOCOLATEYLASTPATHUPDATE: 133426277199629245
CHROME_CRASHPAD_PIPE_NAME: \\.\pipe\crashpad_69092_NFZXZGCTRQNIKKCH
CLION_VM_OPTIONS: C:\Program Files\JetBrains\jetbra\vmoptions\clion.vmoptions
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: AKIS4070
COMSPEC: C:\WINDOWS\system32\cmd.exe
CONDA_DEFAULT_ENV: base
CONDA_EXE: C:\ProgramData\anaconda3\Scripts\conda.exe
CONDA_PREFIX: C:\ProgramData\anaconda3
CONDA_PROMPT_MODIFIER: (base) 
CONDA_P

In [5]:
import yaml
import json
# 打开并读取YAML文件
with open('ToolConfig.yml', 'r', encoding='utf-8') as file:
    config_yml = yaml.load(file, Loader=yaml.FullLoader)

# 打印读取的数据
print(json.dumps(config_yml))

[{"type": "function", "function": {"name": "close_light", "description": "\u5173\u6389\u673a\u7bb1\u7684\u706f"}}]


In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "qwen2:1.5b",
)


In [30]:
from loguru import logger
from tools import tool
import ollama

tools = []
messages = [
        {"role": "system", "content": "你是Alice,是YakumoAki在设计的智能语音助手"}
    ]

import yaml
model_name = 'qwen2.5:1.5b'
    # 打开并读取YAML文件
with open('ToolConfig.yml', 'r', encoding='utf-8') as file:
        tools = yaml.load(file, Loader=yaml.FullLoader)

def chat(msg):
    messages.append({'role': 'user', 'content': msg})

    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools,
    )
    messages.append(response["message"])
    if tool_calls := messages[-1].get("tool_calls", None):
        for tool_call in tool_calls:
            if fn_call := tool_call.get("function"):
                fn_name: str = fn_call["name"]
                fn_args: dict = fn_call["arguments"]
                fn_res: str = globals()(fn_name)(**fn_args)
                messages.append({
                    "role": "tool",
                    "name": fn_name,
                    "content": fn_res,
                })
    response = ollama.chat(
            model=model_name,
            messages=messages,
            tools=tools,
        )
    messages.append(response["message"])
    return messages[:-1]

In [13]:
from loguru import logger
from tools import tool_config
import ollama
class OllamaQW():
    # 加载tool文件中所有工具函数
    import importlib
    module_name = 'tools.tool_list'
    module = importlib.import_module(module_name)
    # 加载工具函数定义
    tools = tool_config.tools
    
    # 模型预设
    base_message = [
            {"role": "system", "content": "你是Alice,是YakumoAki在设计的智能语音助手"}
        ]
    messages = []
    def __init__(self, model_name: str):
        import yaml
        self.model_name = model_name
        logger.info(self.tools)
        
    def chat(self, msg):
        self.messages.append({'role': 'user', 'content': msg})
        logger.info(f"input message: {self.messages[-1]}")
        response = ollama.chat(
            model=self.model_name,
            messages=self.base_message + self.messages[-20:],
            tools=self.tools,
        )
        self.messages.append(response["message"])
        logger.info(f"model response is: {response}")
        if tool_calls := self.messages[-1].get("tool_calls", None):
            for tool_call in tool_calls:
                if fn_call := tool_call.get("function"):
                    fn_name: str = fn_call["name"]
                    fn_args: dict = fn_call["arguments"]
                    logger.info(f"function name: {fn_name}")
                    logger.info(f"function args: {fn_args}")
                    my_function = getattr(self.module, fn_name)
                    fn_res: str = my_function(**fn_args)

                    self.messages.append({
                        "role": "tool",
                        "name": fn_name,
                        "content": fn_res,
                    })
                    logger.info(f"tool response is: {self.messages[-1]}")
            response = ollama.chat(
                model=self.model_name,
                messages=self.base_message + self.messages[-20:],
                tools=self.tools,
            )
            self.messages.append(response["message"])
            logger.info(f"model response is: {response}")
        return self.messages[-1]



qw = OllamaQW(model_name='qwen2.5:3b')
# qw.chat('关了机箱的灯')

2025-01-13 06:22:13.036 | INFO     | __main__:__init__:20 - [{'type': 'function', 'function': {'name': 'close_light', 'description': '关掉机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'open_light', 'description': '打开机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}]


In [12]:
qw.chat('打开机箱的灯')

2025-01-13 06:19:27.336 | INFO     | __main__:chat:24 - input message: {'role': 'user', 'content': '打开机箱的灯'}
2025-01-13 06:19:27.503 | INFO     | __main__:chat:31 - model response is: model='qwen2.5:3b' created_at='2025-01-12T22:19:27.5021813Z' done=True done_reason='stop' total_duration=164421000 load_duration=10788900 prompt_eval_count=188 prompt_eval_duration=2000000 eval_count=16 eval_duration=148000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='open_light', arguments={}))])
2025-01-13 06:19:27.503 | INFO     | __main__:chat:37 - function name: open_light
2025-01-13 06:19:27.504 | INFO     | __main__:chat:38 - function args: {}


AttributeError: module 'tools' has no attribute 'open_light'

In [10]:
qw.chat('close light')

2025-01-13 06:17:48.351 | INFO     | __main__:chat:23 - input message: {'role': 'user', 'content': 'close light'}
2025-01-13 06:17:48.516 | INFO     | __main__:chat:30 - model response is: model='qwen2.5:3b' created_at='2025-01-12T22:17:48.5153795Z' done=True done_reason='stop' total_duration=162822400 load_duration=11395600 prompt_eval_count=244 prompt_eval_duration=6000000 eval_count=16 eval_duration=139000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='close_light', arguments={}))])
2025-01-13 06:17:48.516 | INFO     | __main__:chat:36 - function name: close_light
2025-01-13 06:17:48.517 | INFO     | __main__:chat:37 - function args: {}



执行cmd指令=""C:\Users\Administrator\Downloads\OpenRGB_0.9_Windows_64_b5f46e3\OpenRGB Windows 64-bit\OpenRGB.exe" --profile close"


2025-01-13 06:17:57.671 | INFO     | __main__:chat:46 - tool response is: {'role': 'tool', 'name': 'close_light', 'content': '关了'}
2025-01-13 06:17:57.819 | INFO     | __main__:chat:53 - model response is: model='qwen2.5:3b' created_at='2025-01-12T22:17:57.8182852Z' done=True done_reason='stop' total_duration=145111500 load_duration=10451200 prompt_eval_count=280 prompt_eval_duration=3000000 eval_count=11 eval_duration=123000000 message=Message(role='assistant', content='好的，已经关掉了机箱的灯。', images=None, tool_calls=None)


Message(role='assistant', content='好的，已经关掉了机箱的灯。', images=None, tool_calls=None)

In [14]:
while True:
    msg = input(">: ")
    if msg.lower() == 'exit':
         break
    res = qw.chat(msg)
    print(f"Alice: {res['content']}")

2025-01-13 06:22:32.760 | INFO     | __main__:chat:24 - input message: {'role': 'user', 'content': '你好啊'}
2025-01-13 06:22:32.867 | INFO     | __main__:chat:31 - model response is: model='qwen2.5:3b' created_at='2025-01-12T22:22:32.8661244Z' done=True done_reason='stop' total_duration=103655400 load_duration=11545500 prompt_eval_count=185 prompt_eval_duration=4000000 eval_count=8 eval_duration=86000000 message=Message(role='assistant', content='你好！有什么可以帮助你的吗？', images=None, tool_calls=None)


Alice: 你好！有什么可以帮助你的吗？
